## Agent Chat Lab -  Multi Agent Orchestration

In [1]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

from azure.ai.projects.models import AzureAISearchTool, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from jinja2 import Environment, FileSystemLoader

import asyncio
import random
import time

import tiktoken

import sys
import os
from dotenv import load_dotenv

sys.path.insert(1, "..")
from src.agent import Agent
from src.multiagent import MultiAgent

load_dotenv(override=True)

True

In [2]:
# // Connect to Azure Foundry and extract the AI Search connection id
connection_string = os.environ["PROJECT_CONNECTION_STRING"]
model_deployment_name = os.environ.get("AZURE_OPENAI_4o_MODEL_NAME")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)

# Extract the connection list.
conn_list = project_client.connections._list_connections()["value"]
conn_id = ""
bing_conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    category = conn["properties"].get("category", None)
    if category == "CognitiveSearch":
        conn_id = conn["id"]
    if category == "ApiKey":
        bing_conn_id = conn["id"]

print(f" Search Connection ID: {conn_id}")
print(f" Bing Connection ID: {bing_conn_id}")

 Search Connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/rg-lab-agents/providers/Microsoft.MachineLearningServices/workspaces/project-demo-7zmh/connections/hub-demo-7zmh-connection-AISearch
 Bing Connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/rg-lab-agents/providers/Microsoft.MachineLearningServices/workspaces/project-demo-7zmh/connections/hub-demo-7zmh-connection-BingSearch


In [14]:
# // Name the agents
AGENT_SCORE = "Score"
AGENT_DEBT = "Dividas"
AGENT_CC_SANTANDER = "CC_Santander"
AGENT_CC_WILL = "CC_Will"
AGENT_CC_PICPAY = "CC_PicPay"
AGENT_CC_NEON = "CC_Neon"
AGENT_CC_ITAU = "CC_Itau"
AGENT_CC_DM = "CC_DM"
AGENT_CC_DIGIO = "CC_Digio"
AGENT_BING = "BING"

# // Create the tools
tools = {
    AGENT_SCORE: AzureAISearchTool(index_connection_id=conn_id, index_name="score"),
    AGENT_DEBT: AzureAISearchTool(index_connection_id=conn_id, index_name="debt"),
    AGENT_CC_WILL: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaowill"
    ),
    AGENT_CC_SANTANDER: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaosantander"
    ),
    AGENT_CC_PICPAY: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaopicpay"
    ),
    AGENT_CC_NEON: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaoneon"
    ),
    AGENT_CC_ITAU: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaoitau"
    ),
    AGENT_CC_DM: AzureAISearchTool(index_connection_id=conn_id, index_name="cartaodm"),
    AGENT_CC_DIGIO: AzureAISearchTool(
        index_connection_id=conn_id, index_name="cartaodigio"
    ),
    AGENT_BING: BingGroundingTool(connection_id=bing_conn_id),
}

# // Load Instructions
prompts = Environment(
    loader=FileSystemLoader("../" + os.getenv("TEMPLATE_DIR_PROMPTS"))
)
instructions = prompts.get_template("instructions.jinja")

# // Create agents in Azure AI Foundry

agent_creator = Agent(model_deployment_name, connection_string)


# // Helper function to create agents
async def create_agent(name, tool, tool_type):
    return await agent_creator.create_agent(
        name=name,
        instructions=instructions.render(tool=name, type=tool_type),
        tool={"definitions": tool.definitions, "resources": tool.resources},
    )


agent_1 = await create_agent(AGENT_SCORE, tools[AGENT_SCORE], "Azure AI Search")
agent_2 = await create_agent(AGENT_DEBT, tools[AGENT_DEBT], "Azure AI Search")
agent_3 = await create_agent(
    AGENT_CC_SANTANDER, tools[AGENT_CC_SANTANDER], "Azure AI Search"
)
agent_4 = await create_agent(AGENT_CC_WILL, tools[AGENT_CC_WILL], "Azure AI Search")
agent_5 = await create_agent(AGENT_CC_PICPAY, tools[AGENT_CC_PICPAY], "Azure AI Search")
agent_6 = await create_agent(AGENT_CC_NEON, tools[AGENT_CC_NEON], "Azure AI Search")
agent_7 = await create_agent(AGENT_CC_ITAU, tools[AGENT_CC_ITAU], "Azure AI Search")
agent_8 = await create_agent(AGENT_CC_DM, tools[AGENT_CC_DM], "Azure AI Search")
agent_9 = await create_agent(AGENT_CC_DIGIO, tools[AGENT_CC_DIGIO], "Azure AI Search")
agent_10 = await create_agent(AGENT_BING, tools[AGENT_BING], "Bing Search")

In [ ]:
# // Selection function
selection_template = f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {AGENT_SCORE}
    - {AGENT_DEBT}
    - {AGENT_CC_SANTANDER}
    - {AGENT_CC_WILL}
    - {AGENT_CC_PICPAY}
    - {AGENT_CC_NEON}
    - {AGENT_CC_ITAU}
    - {AGENT_CC_DM}
    - {AGENT_CC_DIGIO}
    - {AGENT_BING}

    If the question is not related to any of the participants, choose {AGENT_BING}.

    History:
    {{{{$history}}}}
"""

termination_template = f"""
    Determine if the question has been answered.  If so, respond with a single word: yes

    History:
    {{{{$history}}}}
"""


count_selection_tokens = len(selection_template)
count_termination_tokens = len(termination_template)

In [52]:
# // Create sample questions
sample_questions = [
    [
        {
            "role": "user",
            "content": "O que é o Feirão Serasa Limpa Nome e como ele funciona?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os descontos de até 99% oferecidos pelo Feirão Serasa Limpa Nome?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso consultar meu CPF pelo site da Serasa?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que é e como funciona o Carrinho de Ofertas para organizar dívidas?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os principais benefícios de negociar suas dívidas com a Serasa?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como confirmar se um canal é oficial do Serasa Limpa Nome?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais as formas de pagamento disponíveis para as negociações no Feirão?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso realizar um atendimento presencial para quitar minha dívida?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais vantagens posso obter utilizando o aplicativo Serasa Limpa Nome?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como funcionam as condições de parcelamento conforme o orçamento oferecidas pela Serasa?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os benefícios do Cartão Will, especialmente sem tarifa e sem anuidade?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso solicitar o Cartão Will através da plataforma do Serasa Crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O Cartão Will oferece cashback; como funciona esse benefício?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais facilidades o Cartão Will proporciona na recarga de celular e no acompanhamento dos gastos?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Existe algum processo 100% digital para solicitar o Cartão Will?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O Cartão Will possui alguma taxa ou anuidade oculta?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são as principais diferenças entre o Cartão Santander Free e os outros cartões Santander?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que torna o Cartão Santander Elite Platinum especial em termos de benefícios?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como o app do Santander permite acompanhar os gastos e controlar o limite do cartão?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que é o Santander AAdvantage Platinum e como ele possibilita o acúmulo de milhas?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso solicitar um cartão de crédito Santander através da plataforma do Serasa Crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais benefícios são oferecidos pelo Cartão Santander Unique?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os benefícios do Cartão PicPay, como anuidade zero e cashback?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como funciona a solicitação do Cartão PicPay pelo Serasa Crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O Cartão PicPay permite parcelar o Pix; como essa funcionalidade opera?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são as diferenças entre o PicPay Card Gold, Platinum e Black?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Que opções o Cartão PicPay oferece para compras online e para a conta digital?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são as principais vantagens do Cartão Neon em comparação com outros cartões de crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como funciona o mecanismo de cashback e o limite elástico do Cartão Neon?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Posso solicitar o Cartão Neon através do Serasa Crédito? Qual o processo?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais benefícios adicionais, como o programa Vai de Visa, o Cartão Neon oferece?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "De que forma o Cartão Neon ajuda no gerenciamento de despesas e na confirmação de limites?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os principais benefícios dos cartões Itaucard oferecidos pelo Itaú?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como funciona o Itaucard Click Platinum e quais são os seus beneficios?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que o Azul Itaucard oferece em termos de acúmulo de pontos e descontos em passagens?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como o Latam PASS Itaucard beneficia os clientes com descontos e acúmulo de milhas?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais vantagens os cartões de crédito Itaú para supermercados (Extra e Pão de Açúcar) proporcionam?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso solicitar um cartão de crédito Itaú através do Serasa Crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os diferenciais do DMcard Mastercard®, como o código de segurança dinâmico?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como funciona o pagamento por aproximação no cartão DMcard?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais recompensas são oferecidas pelo programa Mastercard Surpreenda do DMcard?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Qual é o valor da anuidade do DMcard e como ela é cobrada?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são os benefícios exclusivos do cartão Digio, como o Pix parcelado e o cashback?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como o cartão Digio possibilita compras nacionais e internacionais sem anuidade?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que é o Programa de Pontos Livelo do Digio e como posso acumular pontos?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso gerenciar a fatura do cartão Digio pelo aplicativo?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Quais são as condições para solicitar o cartão Digio pelo Serasa Crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O cartão Digio oferece conversão de moeda para compras internacionais? Quais são as taxas aplicadas?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "O que é o Serasa Score e como ele influencia a obtenção de crédito?",
        },
        {"role": "user", "content": "exit"},
    ],
    [
        {
            "role": "user",
            "content": "Como posso melhorar meu Serasa Score conectando minhas contas e pagando dívidas via Pix?",
        },
        {"role": "user", "content": "exit"},
    ],
]

In [ ]:
# Set the number of requests per minute (N)
maximum_iterations = 1
tokenizer = tiktoken.encoding_for_model("gpt-4o")

async def run_request(conversation):
    responses = []
    # Create a new chat group for each request
    multi_agent = MultiAgent(
        model_deployment_name,
        maximum_iterations,
        selection_template,
        termination_template,
    )

    # Create a chat group with the agents
    chat = multi_agent.create_chat_group(
        agents=[
            agent_1,
            agent_2,
            agent_3,
            agent_4,
            agent_5,
            agent_6,
            agent_7,
            agent_8,
            agent_9,
            agent_10,
        ]
    )
    try:
        for msg in conversation:
            user_input = msg["content"]
            if not user_input:
                continue
            # Handle special commands if present
            if user_input.lower() in ("exit", "reset"):
                await chat.reset()
                break
            # Add the user message to the chat
            await chat.add_chat_message(message=user_input)
            # Invoke the chat and iterate through responses
            metadata = {"selection_tokens": count_selection_tokens, "termination_tokens": count_termination_tokens}
            async for response in chat.invoke():
                # Append the response to the responses list
                responses.append(
                    {
                        "role": response.role,
                        "content": response.content,
                        "tool": response.name,
                        "input_tokens": len(tokenizer.encode(user_input)),
                        "output_tokens": len(tokenizer.encode(response.content)),
                        "metadata": metadata,
                    }
                )
            # End the conversation by adding an exit message
            await chat.add_chat_message(message="exit")
        return responses
    except Exception as e:
        responses.append({"role": e, "content": 0, "tool": 0})
        return responses


Running conversation: [{'role': 'user', 'content': 'Como posso gerenciar a fatura do cartão Digio pelo aplicativo?'}, {'role': 'user', 'content': 'exit'}]
Running conversation: [{'role': 'user', 'content': 'Quais são os benefícios exclusivos do cartão Digio, como o Pix parcelado e o cashback?'}, {'role': 'user', 'content': 'exit'}]
Running conversation: [{'role': 'user', 'content': 'O cartão Digio oferece conversão de moeda para compras internacionais? Quais são as taxas aplicadas?'}, {'role': 'user', 'content': 'exit'}]


In [ ]:
# Schedule N requests in parallel
num_requests = 5
for _ in range(num_requests):
    # Pick a random conversation from the sample list
    responses = []
    tasks = []
    conv = random.choice(sample_questions)
    print(f"Running conversation: {conv}")
    tasks.append(asyncio.create_task(run_request(conv)))

results = await asyncio.gather(*tasks, return_exceptions=True)


responses

[{'role': <AuthorRole.ASSISTANT: 'assistant'>,
  'content': 'Para gerenciar a fatura do seu cartão Digio pelo aplicativo, você pode usufruir de uma função de controle total sobre seus gastos. O aplicativo permite que você visualize a fatura em tempo real, ajudando a acompanhar suas despesas de forma prática e eficiente【3:0†source】.',
  'tool': 'CC_Digio',
  'input_tokens': 14,
  'output_tokens': 61,
  'metadata': {'selection_tokens': 534, 'termination_tokens': 121}},
 {'role': <AuthorRole.ASSISTANT: 'assistant'>,
  'content': 'O cartão Digio oferece conversão de moeda para compras internacionais. As compras feitas em moeda estrangeira são convertidas com base no câmbio do Dólar PTAX (Banco Central) e há uma taxa de conversão de 4% sobre o valor da compra【3:0†source】.',
  'tool': 'CC_Digio',
  'input_tokens': 21,
  'output_tokens': 62,
  'metadata': {'selection_tokens': 534, 'termination_tokens': 121}},
 {'role': <AuthorRole.ASSISTANT: 'assistant'>,
  'content': 'O cartão Digio oferece 

In [ ]:
# // Delete the agents
agent_list = project_client.agents.list_agents().data
for agent in agent_list:
    project_client.agents.delete_agent(agent.id)